In [1]:
import pandas as pd
pd.set_option('precision', 2)
import numpy as np

# custom module
import preprocess

%load_ext watermark
%load_ext autoreload
%autoreload 2
%matplotlib inline

%watermark -d -t -u -v -g -r -b -iv -a "Hongsup Shin" 

Author: Hongsup Shin

Last updated: 2021-03-03 20:04:28

Python implementation: CPython
Python version       : 3.7.9
IPython version      : 7.20.0

Git hash: 56b35164edef17a573db537387e7c2bb5cae037c

Git repo: https://github.com/texas-justice-initiative/officer_involved_shooting_report_2020.git

Git branch: master

pandas: 1.2.2
numpy : 1.20.1



# Prepare the data for the data entry task
## Motivation
We are planning to extract data manually from news articles about OIS incidents. To do so, we will have to look up news articles manually by using information about every incident in the TJI OIS data. Thus, we need to populate the entry spreadsheet Magarita created with existing information.

## Challenge
The level of details in the TJI dataset about the OIS incidents is high. We could add the entire information but this will easily overwhelm human volunteers and also it's going to be more difficult to make manual inputs in the spreadsheet if we have too many columns (you have to scroll left and right a lot).

## Approach
Thus, I'd like to curate the information so that we can transfer useful information for human volunteers and leave granular information out. Of course this does not mean that we are ignoring the information that we are leaving out. The spreadsheet that the volunteers work on can be easily merged with the original TJI dataset.

## Background
There are two TJI officer-involved shooting (OIS) dataset"
- The civlians who are shot by police officers
- The police officers injured or killed in the incidents

For this analysis, the data was downloaded from the TJI website on March 3, 2021 by Hongsup Shin.

In [2]:
# Quick data preprocessing
# loading data
df_cd = pd.read_csv('../Data/Raw/Website/Mar2021/tji_civilians-shot.csv')
df_os = pd.read_csv('../Data/Raw/Website/Mar2021/tji_officers-shot.csv')

# Convert all date columns into the right format
df_cd = preprocess.convert_date_cols(df_cd, 'date')
df_os = preprocess.convert_date_cols(df_os, 'date')

# Select data from 2016 to 2020
years = np.arange(2016, 2021) # this is 2016 to 2020 not 2021
df_cd = df_cd.loc[df_cd['date_incident'].dt.year.isin(years)]; print(df_cd.shape)
df_os = df_os.loc[df_os['date_incident'].dt.year.isin(years)]; print(df_os.shape)

(877, 143)
(167, 47)


## 1. Civilian data

As you can see below, the original data has a lot of columns. If we add all of them, it will be overwhelming.

In [3]:
print(np.sort(df_cd.columns))

['agency_city_1' 'agency_city_10' 'agency_city_11' 'agency_city_2'
 'agency_city_3' 'agency_city_4' 'agency_city_5' 'agency_city_6'
 'agency_city_7' 'agency_city_8' 'agency_city_9' 'agency_county_1'
 'agency_county_10' 'agency_county_2' 'agency_county_3' 'agency_county_4'
 'agency_county_5' 'agency_county_6' 'agency_county_7' 'agency_county_8'
 'agency_county_9' 'agency_email_person_filling_out_1'
 'agency_email_person_filling_out_10' 'agency_email_person_filling_out_11'
 'agency_email_person_filling_out_2' 'agency_email_person_filling_out_3'
 'agency_email_person_filling_out_4' 'agency_email_person_filling_out_5'
 'agency_email_person_filling_out_6' 'agency_email_person_filling_out_7'
 'agency_email_person_filling_out_8' 'agency_email_person_filling_out_9'
 'agency_name_1' 'agency_name_10' 'agency_name_11' 'agency_name_2'
 'agency_name_3' 'agency_name_4' 'agency_name_5' 'agency_name_6'
 'agency_name_7' 'agency_name_8' 'agency_name_9'
 'agency_name_person_filling_out_1' 'agency_name_pe

## Which information in the civilian dataset is going to be copied to the entry sheet?

|Information category| Comprehensive informatin within the category | What will be copied to the spreadsheet|
|---|---|--|
|**Agency** | name, location (city, county, zip code), email address of the peson who filed the report, report date, date that agency received the report, the number of reports filed| name|
| **Civilian** | name, demographics (race (white, black, hispanic, other), gender (male, female), age at the time of incident), whether they were injured or died, date of incident| all |
|**Incident**|time and date, location (city, county, zip code, address), whether civilian carried a deadly weapon, the reason of officers' involvement (the 5 categories the OAG reports use and more details in textual form)| time, date, deadly weapon possession, incident cause|
|**Report**| date that report was filed, whether the custodial death report was filed| none |
|**Media**| names of the news media which covered the incident| all |
|**Officer**| demographics (race, gender, age), details of the officer-caused injury (less than 10% of the data is populated), whether the officer was on duty, whether multiple officers were involved, the number of officers reported| on-duty-ness, mutiple-officer, number of officers |

*The third column is based on my own judgment. So please let me know if you think more information should be added (among the ones in the second column).*

In [4]:
def get_cols_by_substr(df, substr):
    ### find columns in the dataset if they contain the substr
    return df.columns[df.columns.str.contains(substr)]

### Agency names 

Multiple agencies can be involved. There are 11 columns that show agency names. If we add all of them, we will have a lot of redundant information. So below, I am going to merge all 11 columns and only use the unique values of them. 

Example 1
- Before: ['Houston PD', 'Houston PD', 'Houston PD', 'Houston PD']
- After: ['Houston PD']

Example 2:
- Before: ['Houston PD', 'Houston PD', 'Houston PD', 'Austin PD']
- After: ['Houston PD', 'Austin PD']

In [5]:
cols_agency_names = get_cols_by_substr(df_cd, 'agency_name').difference(
    get_cols_by_substr(df_cd, 'agency_name_person_filling_out'))
print(cols_agency_names)

Index(['agency_name_1', 'agency_name_10', 'agency_name_11', 'agency_name_2',
       'agency_name_3', 'agency_name_4', 'agency_name_5', 'agency_name_6',
       'agency_name_7', 'agency_name_8', 'agency_name_9'],
      dtype='object')


In [6]:
df_cd_agency_names_merged = df_cd[cols_agency_names].apply(lambda x: list(set(','.join(x.dropna()).split(','))), axis=1)
df_cd_agency_names_merged = df_cd_agency_names_merged.to_frame()
df_cd_agency_names_merged.columns = ['agency_names']

In [7]:
df_cd_agency_names_merged.head()

,agency_names
66,[SWEETWATER POLICE DEPT]
67,[FORNEY POLICE DEPT]
68,[ABILENE POLICE DEPT]
69,[SAN ANTONIO POLICE DEPT]
70,"[MESQUITE POLICE DEPT, FORNEY POLICE DEPT]"


### Civilian names

There are 3 columns that represent civilian names; first, last and full. We can just use first and last name and drop the full name column because when we merge the first and last names, they are identical to the full name.

In [8]:
civilian_name_merged = df_cd[['civilian_name_first', 'civilian_name_last']].apply(lambda x: ' '.join(x.dropna()), axis=1) 
df_cd.loc[df_cd['civilian_name_full'] != civilian_name_merged, ['civilian_name_first', 'civilian_name_last', 'civilian_name_full']].shape

(210, 3)

**Warning: I found that there are 210 incidents where we don't have victim's names.**

In [9]:
df_cd[['civilian_name_first', 'civilian_name_last', 'civilian_name_full']].isnull().sum()

civilian_name_first    211
civilian_name_last     210
civilian_name_full     210
dtype: int64

### Reordering columns manually for convenience
To minimize the burden of human data collectors, I have reordered the column names so that they can easily look up the incidents on the web.

In [10]:
np.sort(df_cd.columns)

array(['agency_city_1', 'agency_city_10', 'agency_city_11',
       'agency_city_2', 'agency_city_3', 'agency_city_4', 'agency_city_5',
       'agency_city_6', 'agency_city_7', 'agency_city_8', 'agency_city_9',
       'agency_county_1', 'agency_county_10', 'agency_county_2',
       'agency_county_3', 'agency_county_4', 'agency_county_5',
       'agency_county_6', 'agency_county_7', 'agency_county_8',
       'agency_county_9', 'agency_email_person_filling_out_1',
       'agency_email_person_filling_out_10',
       'agency_email_person_filling_out_11',
       'agency_email_person_filling_out_2',
       'agency_email_person_filling_out_3',
       'agency_email_person_filling_out_4',
       'agency_email_person_filling_out_5',
       'agency_email_person_filling_out_6',
       'agency_email_person_filling_out_7',
       'agency_email_person_filling_out_8',
       'agency_email_person_filling_out_9', 'agency_name_1',
       'agency_name_10', 'agency_name_11', 'agency_name_2',
       'agency_

In [11]:
cols_civilian = ['date_incident',
                 'time_incident',
                 'incident_address',
                 'incident_city',
                 'incident_county',
                 'incident_zip',
                 'civilian_name_first', 
                 'civilian_name_last',
                 'civilian_race',
                 'civilian_gender',
                 'civilian_age',
                 'civilian_died',
                 'deadly_weapon',
                 'incident_result_of',
                 'multiple_officers_involved',
                 'num_officers_recorded',
                 'officer_on_duty',
                 'news_coverage_1',
                 'news_coverage_2',
                 'news_coverage_3',
                 'news_coverage_4',
                 'num_rows_about_this_incident']

In [12]:
df_cd_ready = pd.concat([df_cd[cols_civilian], df_cd_agency_names_merged], axis=1)

First 5 example:

In [13]:
df_cd_ready.head()

,date_incident,time_incident,incident_address,incident_city,incident_county,incident_zip,civilian_name_first,civilian_name_last,civilian_race,civilian_gender,...,incident_result_of,multiple_officers_involved,num_officers_recorded,officer_on_duty,news_coverage_1,news_coverage_2,news_coverage_3,news_coverage_4,num_rows_about_this_incident,agency_names
66,2016-01-02,NaN,2000 LAMAR STREET,SWEETWATER,NOLAN,79556.0,LANCE,LIGHT,WHITE,MALE,...,EMERGENCY CALL OR REQUEST FOR ASSISTANCE,True,1,True,KTXS,ABILENE REPORTER NEWS,KTXS,BIG COUNTRY HOMEPAGE,1,[SWEETWATER POLICE DEPT]
67,2016-01-03,NaN,FM 2932 & FM 741,UNINCORPORATED,KAUFMAN,75126.0,CODY,WESBERRY,WHITE,MALE,...,OTHER - SPECIFY TYPE OF CALL,False,1,False,DALLAS MORNING NEWS,NBC DFW,IN FORNEY,FOX 4 NEWS,1,[FORNEY POLICE DEPT]
68,2016-01-04,NaN,1142 BURGER STREET,ABILENE,TAYLOR,79603.0,DEREK LEE,CASEY,BLACK,MALE,...,EMERGENCY CALL OR REQUEST FOR ASSISTANCE,True,2,True,KTXS,ABILENE REPORTER NEWS,BIG COUNTRY HOMEPAGE,LUBBOCK AVALANCHE JOURNAL,1,[ABILENE POLICE DEPT]
69,2016-01-17,NaN,5639 OLD HWY 90 W,SAN ANTONIO,BEXAR,78227.0,ASHTON LANE,MORRIS,WHITE,MALE,...,EMERGENCY CALL OR REQUEST FOR ASSISTANCE,True,1,True,KENS 5,MY SAN ANTONIO,KSAT,NaN,1,[SAN ANTONIO POLICE DEPT]
70,2016-01-19,NaN,1100 RIVERWOOD,DALLAS,DALLAS,75217.0,GARY WAYNE,JONES,WHITE,MALE,...,OTHER - SPECIFY TYPE OF CALL,True,3,True,DFW CBS LOCAL,DALLAS MORNING NEWS,NBC DFW,FOX 4 NEWS,1,"[MESQUITE POLICE DEPT, FORNEY POLICE DEPT]"


In [14]:
df_cd_ready.shape

(877, 23)

## 2. Officer data

The officer data also has a lot of columns. We will need to clean them up a bit.

In [15]:
print(np.sort(df_os.columns))

['agency_city_1' 'agency_city_2' 'agency_county_1' 'agency_county_2'
 'agency_email_person_filling_out_1' 'agency_email_person_filling_out_2'
 'agency_name_1' 'agency_name_2' 'agency_name_person_filling_out_1'
 'agency_name_person_filling_out_2' 'agency_report_date_1'
 'agency_report_date_2' 'agency_zip_1' 'agency_zip_2' 'civilian_age_1'
 'civilian_age_2' 'civilian_age_3' 'civilian_gender_1' 'civilian_gender_2'
 'civilian_gender_3' 'civilian_harm' 'civilian_name_first_1'
 'civilian_name_first_2' 'civilian_name_first_3' 'civilian_name_last_1'
 'civilian_name_last_2' 'civilian_name_last_3' 'civilian_race_1'
 'civilian_race_2' 'civilian_race_3' 'civilian_suicide' 'date_ag_received'
 'date_incident' 'incident_address' 'incident_city' 'incident_county'
 'incident_zip' 'media_link_1' 'media_link_2' 'media_link_3'
 'num_civilians_recorded' 'officer_age' 'officer_gender' 'officer_harm'
 'officer_name_first' 'officer_name_last' 'officer_race']


### Which information in the officer dataset is going to be copied to the entry sheet?

|Information category| Comprehensive informatin within the category | What will be copied to the spreadsheet|
|---|---|--|
|**Agency**|name, location, name of the person who filed the report, report date| name |
|**Officer**|name, demographics (race, gender, age), whether the officer died or got injured| all|
|**Civilian**|name, demographics (race, gender, age), whether the civilian died, got injured, or nothing happened to them, whether the civilian committed suicide| civilian harm, suicide, num of civilians|
|**Incident**| date, location | all|
|**Media**|links to media reports|all|

*The third column is based on my own judgment. So please let me know if you think more information should be added (among the ones in the second column).*

### Agency names
Applying the same logic as the civilian data

In [16]:
cols_agency_names = get_cols_by_substr(df_os, 'agency_name').difference(
    get_cols_by_substr(df_os, 'agency_name_person_filling_out'))
df_os_agency_names_merged = df_os[cols_agency_names].apply(lambda x: list(set(','.join(x.dropna()).split(','))), axis=1)
df_os_agency_names_merged = df_os_agency_names_merged.to_frame()
df_os_agency_names_merged.columns = ['agency_names']

In [17]:
df_os_agency_names_merged.head()

,agency_names
5,[HOUSTON POLICE DEPT]
6,[HOUSTON POLICE DEPT]
7,[MONTGOMERY CO CONST PCT 3]
8,[HOUSTON POLICE DEPT]
9,[EULESS POLICE DEPT]


### Reordering columns manually for convenience

In [18]:
cols_officer = ['date_incident', # time information is include in the date column but not 100% reliable
                'incident_address',
                'incident_city',
                'incident_county',
                'incident_zip',
                'officer_name_first',
                'officer_name_last',
                'officer_race',
                'officer_gender',
                'officer_age',
                'officer_harm',
                'civilian_harm',
                'civilian_suicide',
                'num_civilians_recorded',
                'media_link_1', 
                'media_link_2',
                'media_link_3']

In [19]:
df_os_ready = pd.concat([df_os[cols_officer], df_os_agency_names_merged], axis=1)

In [20]:
df_os_ready.head()

,date_incident,incident_address,incident_city,incident_county,incident_zip,officer_name_first,officer_name_last,officer_race,officer_gender,officer_age,officer_harm,civilian_harm,civilian_suicide,num_civilians_recorded,media_link_1,media_link_2,media_link_3,agency_names
5,2016-01-13,3100 ANITA,HOUSTON,HARRIS,77004,KENNETH,FREGIA,WHITE,MALE,46.0,INJURY,NONE,False,0,CHRONICLE,ABC 13,NaN,[HOUSTON POLICE DEPT]
6,2016-01-19,6011 VAN ZANDT,HOUSTON,HARRIS,77016,JASON,RHODES,WHITE,MALE,33.0,INJURY,NONE,False,1,CHRONICLE,CLICK2HOUSTON,NaN,[HOUSTON POLICE DEPT]
7,2016-02-07,1410 ASHLAND DRIVE,CONROE,MONTGOMERY,77385,NaN,NaN,WHITE,MALE,36.0,INJURY,NONE,False,1,NaN,NaN,NaN,[MONTGOMERY CO CONST PCT 3]
8,2016-02-22,5100 CLOVER,HOUSTON,HARRIS,77021,NaN,NaN,BLACK,MALE,27.0,INJURY,NONE,False,1,CW39,HOUSTONTX.GOV,NaN,[HOUSTON POLICE DEPT]
9,2016-03-01,508 SIMMONS DR.,EULESS,TARRANT,76040,DAVID STEFAN,HOFER,WHITE,MALE,29.0,DEATH,DEATH,False,1,DALLAS MORNING NEWS,NBC DFW,NaN,[EULESS POLICE DEPT]


In [21]:
df_os_ready.shape

(167, 18)

## Export both datasets to csv

In [22]:
df_cd_ready.to_csv('../Data/Preprocessed/Mar2021/civilian_shooting_preprocessed.csv')
df_os_ready.to_csv('../Data/Preprocessed/Mar2021/officer_shooting_preprocessed.csv')